# Homework3

提高2101班杨筠松 U202115980

### 6.1 
基于课本的概念和设计，我们可以得到如下的算法



In [1]:
def itemset_freq_tester(C, X):
    """
    判断项集 X 是否频繁，并返回它的支持度。
    
    参数:
    C - 所有频繁闭合项集及其支持度计数的集合
    X - 要测试的项集
    
    返回值:
    如果项集 X 频繁，则返回其支持度；否则，返回 -1。
    """
    s = None  # 初始化 s 为 None，表示还没有找到包含 X 的频繁闭合项集
    
    # 遍历所有的频繁闭合项集
    for l in C:
        # 检查 X 是否是 l 的子集且 l 是目前找到的最小的频繁闭合项集
        if X.issubset(l) and (s is None or len(l) < len(s)):
            s = l  # 更新 s 为 l
    
    # 如果找到了包含 X 的频繁闭合项集，返回其支持度
    if s is not None:
        return C[s]  # 假设 C 是字典，键为项集，值为支持度
    else:
        # 如果没有找到，返回 -1
        return -1

In [2]:
# 定义频繁闭合项集及其支持度
C = {
    frozenset({'a', 'b', 'c'}): 5,
    frozenset({'a', 'b'}): 7,
    frozenset({'b', 'c'}): 6,
    frozenset({'c'}): 10,
    frozenset({'a'}): 8
}

# 测试项集
test_itemsets = [
    {'a', 'b'}, # 应该返回 7
    {'b'},      # 应该返回最小的包含'b'的项集的支持度，这里是 6
    {'c'},      # 应该返回 10
    {'a', 'c'}, # 应该返回 5
    {'d'},      # 不在 C 中，应该返回 -1
    {'a', 'd'}  # 不在 C 中，应该返回 -1
]

# 执行测试
for test_itemset in test_itemsets:
    support = itemset_freq_tester(C, frozenset(test_itemset))
    print(f"项集 {test_itemset} 的支持度是：{support}")

项集 {'b', 'a'} 的支持度是：7
项集 {'b'} 的支持度是：7
项集 {'c'} 的支持度是：10
项集 {'a', 'c'} 的支持度是：5
项集 {'d'} 的支持度是：-1
项集 {'a', 'd'} 的支持度是：-1


### 6.2 

假设从事务数据库 `D` 中的生成式 `generator`，如果不存在真子集 `Y ⊂ X` 使得 `support(X) = support(Y)`。生成式 `X` 是频繁的生成式，则 `support(X)` 即是最小支持度阈值。设 `G` 是数据库 `D` 所有频繁的生成式的集合。

(a) 仅使用 `G` 和它们的支持度计数，你能确定项集 `A` 是否频繁，并且如果 `A` 频繁，确定 `A` 的支持度吗？如果能，给出你的算法。否则，还需要什么信息？假定有所需要的信息，你能给出一个算法吗？

(b) 闭集频繁项集的概念是什么？

In [4]:
# 假设支持度函数和生成式集合G已经给定
def support(itemset):
    # 这里应该是查询数据库或者计算给定项集的支持度的实际函数
    # 暂时使用一个示例字典来模拟
    example_supports = {
        frozenset({'a'}): 0.6,
        frozenset({'b'}): 0.7,
        frozenset({'c'}): 0.5,
        frozenset({'a', 'b'}): 0.4,
        frozenset({'a', 'c'}): 0.3,
        frozenset({'b', 'c'}): 0.2,
        frozenset({'a', 'b', 'c'}): 0.1
    }
    return example_supports.get(itemset, 0)

# 判断一个项集是否为频繁生成式
def is_freq_generator(A, G):
    A = frozenset(A)
    for gen in G:
        if A == gen or A.issuperset(gen):
            # 判断是否所有真子集的支持度都不等于A的支持度
            if all(support(A) != support(subset) for subset in powerset(A) if subset and subset != A):
                return True, support(A)
    return False, None

# 生成所有真子集的函数
def powerset(iterable):
    from itertools import chain, combinations
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [5]:
# 频繁生成式集合G的例子
G = {
    frozenset({'a'}),
    frozenset({'b'}),
    frozenset({'a', 'b'})
}

# 测试项集A
test_itemsets = [
    {'a'},          # 频繁
    {'a', 'b'},     # 频繁
    {'c'},          # 不频繁
    {'a', 'b', 'c'} # 不频繁
]

# 执行测试
for test_itemset in test_itemsets:
    freq, supp = is_freq_generator(test_itemset, G)
    status = '频繁' if freq else '不频繁'
    print(f"项集 {test_itemset} 是 {status}, 支持度是: {supp}")

项集 {'a'} 是 频繁, 支持度是: 0.6
项集 {'b', 'a'} 是 频繁, 支持度是: 0.4
项集 {'c'} 是 不频繁, 支持度是: None
项集 {'b', 'a', 'c'} 是 频繁, 支持度是: 0.1


### 6.3

使用Apriori算法进行关联规则挖掘的基本步骤。

- (a) 计算频繁项集的所有非空子集一定也是频繁的。
- (b) 计算项集的任意非空子集的支持度计数与该项集的支持度一样大。
- (c) 当定频繁项集I和任意子集s，证明如果“s⇒I\s”的置信度不可能大于“s⇒I”置信度的情况。
- (d) Apriori算法的一种变形考虑到事务数据库D中的事务并不总是等价的。证明在D中频繁的项集是少在D的一个分区中也频繁的。

In [11]:
from itertools import combinations

def get_subsets(itemset):
    """生成给定项集的所有非空子集"""
    subsets = []
    for i in range(1, len(itemset) + 1):
        subsets.extend([set(x) for x in combinations(itemset, i)])
    return subsets

def get_support(itemset, transactions):
    """计算项集在事务数据库中的支持度计数"""
    count = sum(1 for transaction in transactions if itemset.issubset(transaction))
    return count / len(transactions)

def get_confidence(rule, transactions):
    """计算规则的置信度"""
    subset_support = get_support(rule[0], transactions)
    rule_support = get_support(rule[0].union(rule[1]), transactions)
    confidence = rule_support / subset_support if subset_support != 0 else 0
    return confidence

def is_frequent_in_partition(itemset, partition, min_support):
    """验证项集是否在给定分区中是频繁的"""
    support = get_support(itemset, partition)
    return support >= min_support

In [12]:
# 测试事务数据库
transactions = [
    {'a', 'b', 'c'},
    {'a', 'b'},
    {'a', 'c'},
    {'b', 'c'},
    {'a', 'b', 'c', 'd'},
    {'b', 'c', 'd'}
]

# 选择一个频繁项集
frequent_itemset = {'a', 'b', 'c'}

# (a) 验证频繁项集的所有非空子集一定也是频繁的
print("所有非空子集一定也是频繁的:")
for subset in get_subsets(frequent_itemset):
    print(f"{subset} 是频繁的: {get_support(subset, transactions)}")

# (b) 验证项集的任意非空子集的支持度计数
print("\n任意非空子集的支持度计数与项集的支持度一样大:")
for subset in get_subsets(frequent_itemset):
    print(f"{subset} 的支持度: {get_support(subset, transactions)}")

# (c) 比较规则 "s⇒I\s" 和 "s⇒I" 的置信度
s = {'a', 'b'}
I = frequent_itemset
confidence_s_I_minus_s = get_confidence((s, I.difference(s)), transactions)
confidence_s_I = get_confidence((s, I), transactions)
print("\n置信度比较:")
print(f"s⇒I\\s 的置信度: {confidence_s_I_minus_s}")
print(f"s⇒I 的置信度: {confidence_s_I}")

# (d) 验证在事务数据库D中的项集在D的一个分区中也是频繁的
partition = transactions[:3]  # 分区D的前三个事务
print("\n项集在分区中也是频繁的:")
print(f"{frequent_itemset} 在分区中是频繁的: {is_frequent_in_partition(frequent_itemset, partition, 0.5)}")


所有非空子集一定也是频繁的:
{'b'} 是频繁的: 0.8333333333333334
{'a'} 是频繁的: 0.6666666666666666
{'c'} 是频繁的: 0.8333333333333334
{'b', 'a'} 是频繁的: 0.5
{'b', 'c'} 是频繁的: 0.6666666666666666
{'a', 'c'} 是频繁的: 0.5
{'b', 'a', 'c'} 是频繁的: 0.3333333333333333

任意非空子集的支持度计数与项集的支持度一样大:
{'b'} 的支持度: 0.8333333333333334
{'a'} 的支持度: 0.6666666666666666
{'c'} 的支持度: 0.8333333333333334
{'b', 'a'} 的支持度: 0.5
{'b', 'c'} 的支持度: 0.6666666666666666
{'a', 'c'} 的支持度: 0.5
{'b', 'a', 'c'} 的支持度: 0.3333333333333333

置信度比较:
s⇒I\s 的置信度: 0.6666666666666666
s⇒I 的置信度: 0.6666666666666666

项集在分区中也是频繁的:
{'b', 'a', 'c'} 在分区中是频繁的: False


### 6.4
在Apriori算法中，has_infrequent_subset 函数用于剪枝过程。其目的是检查候选项集 
c 是否包含非频繁的子集。如果包含，这个候选项集不可能是频繁的，因此可以被剪掉。根据频繁项集的阿普里奥里（Apriori）属性，如果一个项集是频繁的，那么它的所有子集也一定是频繁的。因此，只有当一个项集的所有子集都是频繁的，这个项集才有可能是频繁的。

In [13]:
def has_infrequent_subset(c, L_k_minus_1):
    """检查候选项集c是否包含非频繁的(k-1)项子集"""
    k_minus_1_subsets = list(itertools.combinations(c, len(c) - 1))
    for subset in k_minus_1_subsets:
        if frozenset(subset) not in L_k_minus_1:
            return True
    return False

在这个函数中，`L_k_minus_1`是前一步找到的所有频繁的 \( k-1 \) 项集的集合。我们通过检查候选项集 `c` 的每个 \( k-1 \) 项子集是否在 `L_k_minus_1` 中来确定 `c` 是否包含非频繁的子集。如果任何一个子集不在 `L_k_minus_1` 中，这意味着这个子集是非频繁的，因此 `c` 也是非频繁的，函数返回 `True`。如果所有的子集都是频繁的，函数返回 `False`，候选项集 `c` 可能是频繁的。


### 6.5
假设：

- 6.3(b): 频繁项集的所有非空子集一定也是频繁的。
- 6.3(c): 关联规则 A → B 的置信度是由 A 支持度和 A ∪ B（即 A 和 B 的并集）的支持度计算出来的。因此，如果 B 不是频繁的，A → B 的置信度将无法满足最小置信度要求。

考虑到这些性质，生成关联规则的有效方法会利用已知的频繁项集以及它们的支持度，而不需要重新计算任何一个子集的支持度。这种方法会使用一个由单个项逐步构建的过程，每次只考虑那些已知为频繁的项集作为规则的右侧（后件），这样可以确保规则的置信度计算是基于有效信息的。

In [19]:
from itertools import combinations

def generate_association_rules(frequent_itemsets, min_confidence):
    rules = []
    for I, support_I in frequent_itemsets.items():
        # 从1开始，因为至少需要一个元素的子集到len(I)-1，因为我们不想要整个集合
        for i in range(1, len(I)):
            for A in combinations(I, i):
                A = frozenset(A)
                B = I - A
                confidence = support_I / frequent_itemsets[A]
                if confidence >= min_confidence:
                    rules.append((A, B, confidence, support_I))
    return rules

In [20]:
# 示例数据和测试
frequent_itemsets = {
    frozenset({'milk', 'bread'}): 0.4,
    frozenset({'bread'}): 0.5,
    frozenset({'milk'}): 0.6
}

rules = generate_association_rules(frequent_itemsets, 0.7)
for rule in rules:
    print(f"规则: {set(rule[0])} -> {set(rule[1])}, 置信度: {rule[2]:.2f}, 支持度: {rule[3]:.2f}")

规则: {'bread'} -> {'milk'}, 置信度: 0.80, 支持度: 0.40
